In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 第一次執行使用
!unzip "/content/drive/MyDrive/Colab Notebooks/CIFAR10/CIFAR10.zip"

串流輸出內容已截斷至最後 5000 行。
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000001.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000006.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000015.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000030.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000042.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000063.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000068.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000074.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000075.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000076.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000079.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000080.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000083.png  
  inflating: CIFAR10/train/truck/aerial_ladder_truck_s_000087.png  
  inflating: CIFAR10/train/

In [4]:
# 定義train與test資料集路徑
train_path = "/content/CIFAR10/train"
test_path = "/content/CIFAR10/test"

In [5]:
# 定義keras內建之ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode = 'nearest',
    validation_split=0.2
)
test_generator = ImageDataGenerator(rescale=1./255)

In [6]:
# 取得train/val/test資料
train_data = train_generator.flow_from_directory(
    train_path, subset='training', target_size=(32, 32), batch_size=64, class_mode='categorical', shuffle=True
)
val_data = train_generator.flow_from_directory(
    train_path, subset='validation', target_size=(32, 32), batch_size=64, class_mode='categorical', shuffle=True
)
test_data = test_generator.flow_from_directory(
    test_path, target_size=(32, 32), batch_size=64, class_mode='categorical', shuffle=True
)

Found 40000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


In [8]:
# 建立CNN模型
from tensorflow.keras import datasets, layers, models
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10762987136523700354
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11154422528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3397329791325563264
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
____________________________

In [9]:
# 定義f1算法
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
# 初始化優化器並設定績效計算與callback等方法
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True,
    verbose=0,
)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(train_data, epochs=200, validation_data=val_data, callbacks=[early_stop], verbose=1)

Epoch 1/200
625/625 [==============================] - 55s 85ms/step - loss: 2.0412 - accuracy: 0.2242 - f1_m: 0.0162 - precision_m: 0.1876 - recall_m: 0.0087 - val_loss: 1.7579 - val_accuracy: 0.3236 - val_f1_m: 0.1143 - val_precision_m: 0.5044 - val_recall_m: 0.0652
Epoch 2/200
625/625 [==============================] - 52s 83ms/step - loss: 1.7002 - accuracy: 0.3727 - f1_m: 0.1586 - precision_m: 0.6183 - recall_m: 0.0928 - val_loss: 1.6479 - val_accuracy: 0.3756 - val_f1_m: 0.2369 - val_precision_m: 0.5354 - val_recall_m: 0.1535
Epoch 3/200
625/625 [==============================] - 52s 83ms/step - loss: 1.5927 - accuracy: 0.4154 - f1_m: 0.2487 - precision_m: 0.6352 - recall_m: 0.1568 - val_loss: 1.6029 - val_accuracy: 0.4028 - val_f1_m: 0.2527 - val_precision_m: 0.5727 - val_recall_m: 0.1634
Epoch 4/200
625/625 [==============================] - 51s 82ms/step - loss: 1.5176 - accuracy: 0.4488 - f1_m: 0.3079 - precision_m: 0.6589 - recall_m: 0.2038 - val_loss: 1.5409 - val_accuracy:

In [11]:
# 測試模型績效
loss, accuracy, f1_score, precision, recall = model.evaluate(test_data)

157/157 [==============================] - 4s 25ms/step - loss: 0.8208 - accuracy: 0.7281 - f1_m: 0.7258 - precision_m: 0.7984 - recall_m: 0.6663


In [12]:
model.save('/content/drive/MyDrive/Colab Notebooks/CIFAR10/model_02')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/CIFAR10/model_01/assets


In [18]:
 ##### 下一個模型測試，測試將relu換為tanh是否提升績效 #####

In [13]:
# 建立CNN模型
from tensorflow.keras import datasets, layers, models
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='tanh'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='tanh'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8055188231187055878
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11154422528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10342074039120666572
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
__________________________

In [14]:
# 定義f1算法
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [15]:
# 初始化優化器並設定績效計算與callback等方法
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True,
    verbose=0,
)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(train_data, epochs=200, validation_data=val_data, callbacks=[early_stop], verbose=1)

Epoch 1/200
625/625 [==============================] - 56s 87ms/step - loss: 1.9518 - accuracy: 0.2799 - f1_m: 0.0677 - precision_m: 0.4114 - recall_m: 0.0380 - val_loss: 1.7667 - val_accuracy: 0.3344 - val_f1_m: 0.1663 - val_precision_m: 0.5008 - val_recall_m: 0.1008
Epoch 2/200
625/625 [==============================] - 54s 86ms/step - loss: 1.7187 - accuracy: 0.3757 - f1_m: 0.1845 - precision_m: 0.5859 - recall_m: 0.1111 - val_loss: 1.6506 - val_accuracy: 0.3880 - val_f1_m: 0.2005 - val_precision_m: 0.5811 - val_recall_m: 0.1224
Epoch 3/200
625/625 [==============================] - 54s 86ms/step - loss: 1.6233 - accuracy: 0.4063 - f1_m: 0.2481 - precision_m: 0.6306 - recall_m: 0.1562 - val_loss: 1.6749 - val_accuracy: 0.3693 - val_f1_m: 0.2033 - val_precision_m: 0.5733 - val_recall_m: 0.1246
Epoch 4/200
625/625 [==============================] - 54s 87ms/step - loss: 1.5754 - accuracy: 0.4285 - f1_m: 0.2770 - precision_m: 0.6466 - recall_m: 0.1785 - val_loss: 1.6000 - val_accuracy:

In [16]:
# 測試模型績效
loss, accuracy, f1_score, precision, recall = model.evaluate(test_data)

157/157 [==============================] - 4s 24ms/step - loss: 1.1192 - accuracy: 0.6204 - f1_m: 0.5974 - precision_m: 0.7197 - recall_m: 0.5119


In [17]:
model.save('/content/drive/MyDrive/Colab Notebooks/CIFAR10/model_01')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/CIFAR10/model_01/assets


In [ ]:
 ##### 下一個模型測試，測試將relu換為selu是否提升績效 #####

In [10]:
# 建立CNN模型
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='selu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3, 3), activation='selu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3, 3), activation='selu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='selu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6443234468812844980
]
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
______

In [11]:
# 定義f1算法
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
# 初始化優化器並設定績效計算與callback等方法
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=30,
    restore_best_weights=True,
    verbose=0,
)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy', f1_m, precision_m, recall_m],
)

history = model.fit(train_data, epochs=200, validation_data=val_data, callbacks=[early_stop], verbose=1)

Epoch 1/200
625/625 [==============================] - 70s 111ms/step - loss: 2.0250 - accuracy: 0.2731 - f1_m: 0.0735 - precision_m: 0.4570 - recall_m: 0.0416 - val_loss: 1.7971 - val_accuracy: 0.3322 - val_f1_m: 0.1619 - val_precision_m: 0.5104 - val_recall_m: 0.0974
Epoch 2/200
625/625 [==============================] - 68s 108ms/step - loss: 1.7418 - accuracy: 0.3607 - f1_m: 0.1845 - precision_m: 0.5670 - recall_m: 0.1117 - val_loss: 1.7192 - val_accuracy: 0.3681 - val_f1_m: 0.2226 - val_precision_m: 0.5159 - val_recall_m: 0.1432
Epoch 3/200
625/625 [==============================] - 68s 109ms/step - loss: 1.6720 - accuracy: 0.3910 - f1_m: 0.2298 - precision_m: 0.6022 - recall_m: 0.1437 - val_loss: 1.6458 - val_accuracy: 0.4032 - val_f1_m: 0.2455 - val_precision_m: 0.5922 - val_recall_m: 0.1563
Epoch 4/200
625/625 [==============================] - 68s 109ms/step - loss: 1.6016 - accuracy: 0.4163 - f1_m: 0.2751 - precision_m: 0.6173 - recall_m: 0.1791 - val_loss: 1.6393 - val_accur

In [13]:
# 測試模型績效
loss, accuracy, f1_score, precision, recall = model.evaluate(test_data)

157/157 [==============================] - 5s 31ms/step - loss: 0.8650 - accuracy: 0.7177 - f1_m: 0.7163 - precision_m: 0.8019 - recall_m: 0.6484


In [14]:
model.save('/content/drive/MyDrive/Colab Notebooks/CIFAR10/model_03')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/CIFAR10/model_03/assets
